### AI agents are the programs where the LLMs control the workflow.

#### AI Agent: 
#### → Goal-oriented reasoning 
#### → Decides its own next steps 
#### → Adapts to unexpected situations 
#### → Actually thinks through problems

In [ ]:
from agents import Agent, Runner, guardrail
from  dotenv  import load_dotenv
import asyncio 
from IPython.display import display, Markdown
from pydantic import BaseModel 

In [ ]:
# loading the env variable i.e openai_api_key
# keep env var name saved as OPENAI_API_KEY

load_dotenv(override=True)

True

In [ ]:
# Building our first agent - A finance tutor

finance_tutor = Agent(
    name="Finance tutor",
    instructions='''You are a finance tutor. Your job is to tackle all finance related questions. 
    If user asks, anything unrealted to finance, kindly let them know, it's out of your scope.''',
    model="gpt-4o-mini",
)

#### A quck summary of sync vs async

Sync (Synchronous)

* Tasks run one after another, blocking until each finishes.

* If one API call takes 10s, the next starts only after that 10s ends.

* Good for simple scripts but bad for scaling concurrent users.

Async (Asynchronous)

* Tasks run concurrently, overlapping wait times (I/O, API calls, Disk operations).

* 100 calls that each take 10s can all finish in ~10s instead of ~1,000s.

* Used in AI agents to handle many API calls, tools, and user requests at once without blocking the agent’s reasoning loop.

In OpenAI Agents SDK, async is crucial because agents often:

* Call the OpenAI API multiple times per step,

* Invoke external tools/APIs,

* And need to serve multiple users concurrently — async keeps everything responsive.

In [19]:
query = "What are some insightful trends in finance?" 
results = await asyncio.gather(Runner.run(finance_tutor, input=query)) 
outputs = [result.final_output for result in results]
print(outputs[0])

Here are some insightful trends in finance:

1. **Digital Transformation**: The shift towards digital platforms for banking, investing, and trading continues to grow, with fintech companies leveraging technology to offer innovative solutions.

2. **Sustainable Investing**: There is increasing interest in Environmental, Social, and Governance (ESG) factors, with more investors seeking to align their investments with their values and focusing on sustainable companies.

3. **Decentralized Finance (DeFi)**: The rise of blockchain technology has led to an increase in decentralized finance platforms, allowing users to access financial services without traditional intermediaries.

4. **Artificial Intelligence and Machine Learning**: Financial institutions are increasingly employing AI and ML for risk assessment, fraud detection, personalized banking, and trading strategies.

5. **Regulatory Changes**: With growing scrutiny on financial practices, regulatory frameworks are evolving, impacting 

In [ ]:
# Display the output as rendered markdown in the notebook
display(Markdown(outputs[0]))

Here are some insightful trends in finance:

1. **Digital Transformation**: The shift towards digital platforms for banking, investing, and trading continues to grow, with fintech companies leveraging technology to offer innovative solutions.

2. **Sustainable Investing**: There is increasing interest in Environmental, Social, and Governance (ESG) factors, with more investors seeking to align their investments with their values and focusing on sustainable companies.

3. **Decentralized Finance (DeFi)**: The rise of blockchain technology has led to an increase in decentralized finance platforms, allowing users to access financial services without traditional intermediaries.

4. **Artificial Intelligence and Machine Learning**: Financial institutions are increasingly employing AI and ML for risk assessment, fraud detection, personalized banking, and trading strategies.

5. **Regulatory Changes**: With growing scrutiny on financial practices, regulatory frameworks are evolving, impacting compliance, transparency, and consumer protection.

6. **Remote Work and Virtual Services**: The pandemic has accelerated the adoption of remote work in finance, with many services now offered virtually, changing the client-service provider interactions.

7. **Cryptocurrency Adoption**: More businesses and institutions are accepting cryptocurrencies, and central banks are exploring Central Bank Digital Currencies (CBDCs) as a response to the changing landscape.

8. **Robo-Advisors Growth**: Automated investment platforms are gaining popularity, especially among younger investors who prefer low-cost, hands-off investment strategies.

9. **Data Analytics**: The use of big data analytics is becoming commonplace, driving decision-making in investment strategies and customer engagement.

10. **Focus on Diversity and Inclusion**: There’s a growing emphasis on diversity, equity, and inclusion within financial institutions and investment practices, aiming to create a more balanced representation in the industry.

These trends reflect the evolving landscape of finance, driven by technology, social responsibility, and changing consumer behaviors.

### Multi agent architectures by role

When you have multiple agents working together, their roles differ:

**Manager (Coordinator / Orchestrator Agent)**

Supervises other agents, delegates tasks, monitors progress.

Example: A "Project Manager Agent" that breaks down a research problem into subtasks and assigns them to worker agents.

**Worker (Executor Agent / Specialist Agent)**

Focused on a single domain or task, usually follows instructions from a manager or user.

Example: A "Python Coder Agent" that only writes and executes code.

**Peer-to-Peer Agents**

No hierarchy, agents collaborate as equals, share knowledge, and negotiate outcomes.

Example: A group of research bots discussing different scientific papers and converging on insights.

**Broker / Mediator Agent**

Acts as an interface between multiple agents or between agents and external systems (APIs, databases).

Example: An API-bridge agent that translates user requests into calls to other services.

**Memory / Knowledge Agents**

Dedicated to storing, retrieving, and updating long-term memory or contextual knowledge.

Example: A "Knowledge Hub Agent" that maintains embeddings and surfaces relevant context to other agents.


### By Behavior / Function

Based on how the agent interacts with its environment:

**Reactive Agents**

Act directly based on stimuli, without planning.

Example: A stock-trading bot reacting instantly to market signals.

**Deliberative / Planning Agents**

Use reasoning, plan ahead, consider multiple outcomes before acting.

Example: An AI travel planner that compares multiple routes before deciding.

**Learning Agents**

Improve with time using feedback, reinforcement learning, or fine-tuning.

Example: A game-playing AI that gets better by self-play.

**Conversational Agents**

Focused on natural language interactions (chatbots, copilots).

######################################################

### Multi step agent with handoffs 

A manager/orchestrating agent that uses 2 other agents to handoff tasks. 

In [42]:
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent]
)


In [46]:

result_history  = await asyncio.gather(Runner.run(triage_agent, input="Who was nicola tesla"))
result_history = [result.final_output for result in result_history]
display(Markdown(result_history[0]))

Nikola Tesla was a Serbian-American inventor, electrical engineer, mechanical engineer, and futurist, best known for his contributions to the development of alternating current (AC) electricity systems.

**Key facts about Nikola Tesla:**
- **Birth:** July 10, 1856, in Smiljan, Austrian Empire (now Croatia)
- **Death:** January 7, 1943, in New York City

**Major Contributions:**
- Developed the AC (alternating current) system for electric power transmission, which is the standard used today.
- Invented the Tesla coil, a high-voltage transformer used in radio technology and experiments.
- Contributed to the development of radio, X-rays, radar, and wireless transmission of energy.
- Held over 300 patents for his inventions and designs.

**Other Notes:**
- Tesla worked for a time with Thomas Edison but later became a rival, mainly because Edison supported direct current (DC) while Tesla advocated for AC.
- He was known for his visionary ideas, some of which were ahead of his time and were not practical in his day.
- Tesla died relatively poor and alone, but today he is celebrated as one of the great inventors and minds in science and engineering.

If you want to know more about a specific invention or part of Tesla's life, let me know!

### Guardrails 

In AI agents, guardrails are rules, constraints, or validations that ensure the agent’s outputs or behaviors stay within safe, correct, and expected boundaries.

Think of them like:

* A seatbelt for LLM outputs (safety).

* A linter/validator for structured responses (correctness).

* A policy filter for compliance (boundaries).

#### Guardrails do things like:

* Prevent hallucinations or harmful instructions.

* Ensure the model outputs valid JSON / schema-compliant data.

* Restrict answers to a certain domain or vocabulary.

* Stop the model from calling dangerous tools with invalid parameters.

#### How do we enforce Guardrails?

**Schema validation (most common):**

Use Pydantic or JSON schema to ensure outputs are correctly formatted.

**Rule-based filters:**

Block toxic, unsafe, or disallowed outputs (keywords, regex).

**Content policies:**

Example: "Never give medical diagnosis, only general advice."

**Control flows / workflows:**

Prevent the agent from looping infinitely or making irrelevant tool calls.

### Pydantic models - Used for type hinting and data validation 
* Super important to validate outputs and inputs in a multi-agent network 

############## Example 1###############

In [50]:
# snippet to explain pydantic models for data validation 

from pydantic import BaseModel, Field, ValidationError

class WeatherReport(BaseModel):
    location: str = Field(..., description="City name")
    temperature_c: float = Field(..., ge=-100, le=100, description="Temperature in Celsius")
    condition: str = Field(..., description="Weather condition, e.g. 'sunny', 'rainy'")

# Example: valid output
data = {"location": "Delhi", "temperature_c": 32.5, "condition": "Sunny"}
report = WeatherReport(**data)
print(report)

# Example: invalid output (too hot!)
try:
    bad_data = {"location": "Delhi", "temperature_c": 300, "condition": "Sunny"}
    WeatherReport(**bad_data)
except ValidationError as e:
    print("❌ Guardrail triggered:", e)


location='Delhi' temperature_c=32.5 condition='Sunny'
❌ Guardrail triggered: 1 validation error for WeatherReport
temperature_c
  Input should be less than or equal to 100 [type=less_than_equal, input_value=300, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal


############## Example 2 ############

In [ ]:

class report_validator(BaseModel):
    stock_price : float = Field(le=5000, ge=500, description="Stock price value guardrails")
    year_period : int = Field(le=2024, ge=2019, description="Year period as a datetime object between 2019 and 2024")

    
try:
    data = {"stock_price": 500.23,"year_period": 2023 }
    
    report_validator(**data)
    print("Validated data:", data)
except ValidationError as e: 
    print("ValidationError", e)   

Validated data: {'stock_price': 500.23, 'year_period': 2023}


############# Example 3 ####################

In [109]:
from pydantic import BaseModel, field_validator, ValidationError
import math

class pulse(BaseModel):
    pulse_type: bool
    pulse_value: int

    # Custom validation logic 
    @field_validator("pulse_value")
    def validate_pulse_value(cls, val):
        # Check if sqrt(val) is even
        if math.sqrt(val) % 2 == 0:
            print("Even signal -verified")
            return val
        # If not, raise a ValueError so Pydantic triggers a validation error
        raise ValueError(f"pulse_value {val} is invalid: sqrt({val}) % 2 != 0")


In [110]:
# Prepare test data for the pulse model
try: 
    data = {
    "pulse_type": False, 
    "pulse_value": 144
    }
    print(pulse(**data))

except ValidationError as e: 
    print(e)


Even signal -verified
pulse_type=False pulse_value=144


#### Pydantci JSON serialization - ability to convert pydantic models to JSON 

In [ ]:
# pydantic model
print(type(pulse)) 

# JSON serialization 
pulse1 = pulse(pulse_type=True, pulse_value=144)
print(pulse1.model_dump_json())

# For example:
p = pulse(pulse_type=True, pulse_value=144)
print(p.model_dump_json())


<class 'pydantic._internal._model_construction.ModelMetaclass'>
Even signal -verified
{"pulse_type":true,"pulse_value":144}
Even signal -verified
{"pulse_type":true,"pulse_value":144}


In [119]:
pulse(pulse_type=True, pulse_value=144).model_dump()

Even signal -verified


{'pulse_type': True, 'pulse_value': 144}

#############################################

In [123]:
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner
from agents.exceptions import InputGuardrailTripwireTriggered
from pydantic import BaseModel
import asyncio

class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

# content flow guardrail agent 
# here the output type is the pydantic model i.e we expect output to be in pydantic model's shape. 
guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
) 

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)


async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    print("Context:",ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

triage_agent = Agent( 
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
)

async def main():
    # Example 1: History question
    try:
        result = await Runner.run(triage_agent, "who was the first president of the united states?")
        print(result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print("Guardrail blocked this input:", e)

    # Example 2: General/philosophical question
    try:
        result = await Runner.run(triage_agent, "What is the meaning of life?")
        print(result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print("Guardrail blocked this input:", e)

# In Jupyter or other environments with a running event loop, use nest_asyncio or asyncio.create_task
import sys

def run_async(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = None

    if loop and loop.is_running():
        # If we're in a running event loop (e.g., Jupyter), use create_task and gather
        import nest_asyncio
        nest_asyncio.apply()
        task = asyncio.ensure_future(coro)
        return task
    else:
        return asyncio.run(coro)

if __name__ == "__main__":
    run_async(main())

Context: None
The first president of the United States was George Washington. He served as president from 1789 to 1797 and is widely recognized as one of the Founding Fathers of the country. Washington was unanimously elected as the nation’s first president after leading the American colonies to victory over Great Britain in the Revolutionary War and presiding over the Constitutional Convention in 1787. His leadership helped shape the office of the presidency and set many important precedents for future presidents.
Context: None
Guardrail blocked this input: Guardrail InputGuardrail triggered tripwire
